In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

2025-03-18 10:17:48.056714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-18 10:17:48.056881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-18 10:17:48.238137: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
data = pd.read_csv("/kaggle/input/diabetic-retinopathy-resized/trainLabels_cropped.csv")
print(data.value_counts().sum())
print(data['level'].unique())
print(data['level'].value_counts().unique())
data.head()

35108
[0 1 2 4 3]
[25802  5288  2438   872   708]


,Unnamed: 0.1,Unnamed: 0,image,level
0,0,0,10_left,0
1,1,1,10_right,0
2,2,2,13_left,0
3,3,3,13_right,0
4,4,4,15_left,1


In [5]:
data.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)
data.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [6]:
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

# Define your dataset
X = data['image']  # Your feature data
y = data['level']  # Your class labels

# Convert DataFrame or Series to a numpy array
X_array = X.values.reshape(-1,1)
y_array = y.values.reshape(-1,1)

# Count the number of datapoints for each class
class_counts = {0: 25802, 1: 5288, 2: 2438, 3: 708, 4: 872}

# Find the target number of datapoints
target_count = class_counts[3]

# Undersample the majority classes
undersampler = RandomUnderSampler(sampling_strategy={label: target_count for label in class_counts.keys() if label != 3})
X_resampled, y_resampled = undersampler.fit_resample(X_array, y_array)

# Check the new class counts
unique, counts = np.unique(y_resampled, return_counts=True)
resampled_class_counts = dict(zip(unique, counts))
print("Resampled class counts:", resampled_class_counts)

Resampled class counts: {0: 708, 1: 708, 2: 708, 3: 872, 4: 708}


In [9]:
new_X = []
for i in X_resampled:
  for j in i:
    new_X.append(j)

In [10]:
y_resampled

array([0, 0, 0, ..., 4, 4, 4])

In [11]:
# Create a new dataframe with resampled data
resampled_data = {'image': new_X,
        'level': y_resampled}

balanced_data = pd.DataFrame(resampled_data)

# Display the first few rows of the new dataframe
balanced_data.sample(10)

,image,level
1820,30959_left,2
3360,4005_left,4
1862,37771_left,2
458,13167_left,0
987,20280_right,1
2692,29018_right,3
2187,3422_left,3
588,32903_left,0
2133,509_left,3
3410,18902_right,4


In [12]:
print(balanced_data.value_counts().sum())
print(balanced_data['level'].unique())
print(balanced_data['level'].value_counts().unique())

3704
[0 1 2 3 4]
[872 708]


In [14]:
balanced_data['image'] = balanced_data['image']+'.jpeg'
balanced_data.head()

,image,level
0,17638_left.jpeg,0
1,38046_left.jpeg,0
2,25112_left.jpeg,0
3,8334_left.jpeg,0
4,2191_right.jpeg,0


In [15]:
train, val = train_test_split(balanced_data, test_size=0.2, random_state=42)

In [16]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

Found 2963 validated image filenames.
Found 741 validated image filenames.


In [21]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [22]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [23]:
model = Model(inputs=base_model.input, outputs=predictions)

In [24]:
for layer in base_model.layers:
    layer.trainable = False

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(train_generator, validation_data=val_generator, epochs=40)

Epoch 1/40


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 101s 983ms/step - accuracy: 0.3095 - loss: 1.9084 - val_accuracy: 0.3738 - val_loss: 1.3779
Epoch 2/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 132s 897ms/step - accuracy: 0.4262 - loss: 1.3162 - val_accuracy: 0.4035 - val_loss: 1.3671
Epoch 3/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 145s 934ms/step - accuracy: 0.4652 - loss: 1.2556 - val_accuracy: 0.3954 - val_loss: 1.4119
Epoch 4/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 86s 901ms/step - accuracy: 0.4558 - loss: 1.2035 - val_accuracy: 0.3846 - val_loss: 1.4451
Epoch 5/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 86s 908ms/step - accuracy: 0.5121 - loss: 1.1516 - val_accuracy: 0.4049 - val_loss: 1.3703
Epoch 6/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 85s 898ms/step - accuracy: 0.5230 - loss: 1.1255 - val_accuracy: 0.4022 - val_loss: 1.3721
Epoch 7/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 87s 914ms/step - accuracy: 0.5506 - loss: 1.0718 - val_accuracy: 0.4507 - val_loss: 1.3701
Epoch 8/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 90s 944ms/step - accuracy: 0.5593 - loss: 1.0265 - val_accuracy: 0.

In [31]:
model.save('drdMbNetV2_40epochs_97.h5')

In [19]:
!pip show tensorflow

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, tf_keras, witwidget


In [30]:
!pip show keras

Name: keras
Version: 3.1.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: keras-tuner, tensorflow


In [32]:
!pip show numpy

Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are
        met:
        
            * Redistributions of source code must retain the above copyright
               notice, this list of conditions and the following disclaimer.
        
            * Redistributions in binary form must reproduce the above
               copyright notice, this list of conditions and the following
               disclaimer in the documentation and/or other materials provided
               with the distribution.
        
            * Neither the name of the NumPy Developers nor the names of any
               contributors may be used to endorse or promo

In [33]:
!pip show pandas

Name: pandas
Version: 2.2.1
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials 

In [ ]:
!pip show 